# Using an output handler for validating agent's output

As no one can guarantee the validity or integrity of a raw LLM output, getting a sensible and/or structured output from an agent is tricky. In motleycrew, we addressed this issue with a novel approach: please welcome **output handlers**.

As of now, tool calling is the most reliable way to get a structured output from an LLM. So why not have an output handler - a regular tool with an input schema and a description which instructs the agent to return the output only via this tool. Moreover, we can force the agent into doing this: if it attempts to finish the regular way, we intercept the message and remind the agent that it must call the tool instead.

In [1]:
from langchain_community.tools import DuckDuckGoSearchRun

from motleycrew import MotleyCrew
from motleycrew.agents.langchain import ReActToolCallingMotleyAgent
from motleycrew.common import configure_logging
from motleycrew.tasks import SimpleTask

from motleycrew.common.exceptions import InvalidOutput

from motleycrew.tools import MotleyTool

We'll be prompting our agent to create a report on the latest advancements in AI.  
Say we want it to also include some info on AI applications in physics if it did not already. So our output handler will look for the keyword "physics" in the agent's output.

Also, we can do any data manipulations we want inside the output handler. In this case, let's just place the output into a certain field.

In [2]:
class OutputHandler(MotleyTool):
    def run(self, output: str):
        if "physics" not in output.lower():
            raise InvalidOutput("Add more information about AI applications in physics.")

        return {"checked_output": output}

output_handler = OutputHandler(
    name="output_handler",
    description="Output handler",
    return_direct=True,  # This will force the tool to return its output directly to the user
)  # Schema is inferred from the `run` method signature, but you can also specify it explicitly

Now let's create our agent and launch the crew!

In [3]:
search_tool = DuckDuckGoSearchRun()
researcher = ReActToolCallingMotleyAgent(
    tools=[search_tool, output_handler],
    verbose=True,
    chat_history=True,
    force_output_handler=True,  # This will force the agent to use the output handler instead of regular output
)

crew = MotleyCrew()
task = SimpleTask(
    crew=crew,
    name="produce comprehensive analysis report on AI advancements in 2024",
    description="""Conduct a comprehensive analysis of the latest advancements in AI in 2024.
Identify key trends, breakthrough technologies, and potential industry impacts.
Perform no more than 3 search queries. Your final answer MUST be a full analysis report""",
    agent=researcher,
)

In [4]:
crew.run()



> Entering new AgentExecutor chain...

Invoking: `duckduckgo_search` with `{'query': 'latest advancements in AI 2024'}`
responded: Thought: To conduct a comprehensive analysis of the latest advancements in AI in 2024, I need to gather information on key trends, breakthrough technologies, and potential industry impacts. I will perform three search queries to collect this information. The first query will focus on general advancements in AI in 2024, the second on breakthrough technologies, and the third on industry impacts.

I will start with the first query to get an overview of the latest advancements in AI in 2024.

Calling the tool to perform the first search query.



What's next for AI in 2024 The Top 5 Artificial Intelligence (AI) Trends For 2024. Adobe Stock. It's been a year since OpenAI released ChatGPT, opening the door to seamlessly weave AI into the fabric of our daily lives ... AI for everything: 10 Breakthrough Technologies 2024 10 top AI and machine learning trends for 


Invoking: `output_handler` with `{'output': "**Comprehensive Analysis of AI Advancements in 2024**\n\n**1. Key Trends in AI for 2024:**\n- **Generative AI:** The release of advanced generative AI models, such as OpenAI's ChatGPT, has revolutionized how AI is integrated into daily life. These models are becoming more sophisticated, enabling more natural and context-aware interactions.\n- **AI in Healthcare:** AI is making significant strides in healthcare, with applications ranging from diagnostics to personalized treatment plans. AI-driven tools are improving patient outcomes and streamlining healthcare operations.\n- **AI for Sustainability:** AI technologies are being leveraged to address environmental challenges. This includes optimizing energy consumption, improving waste management, and supporting conservation efforts.\n- **Edge AI:** The trend towards edge computing is growing, with AI models being deployed on devices closer to the data source. This reduces latency and enhances 

[TaskUnit(status=done)]

As you can see in the logs, our physics requirement was successfully fed back to the agent!

In [5]:
from IPython.display import display, Markdown

final_result = task.output["checked_output"]
display(Markdown(final_result))

**Comprehensive Analysis of AI Advancements in 2024**

**1. Key Trends in AI for 2024:**
- **Generative AI:** The release of advanced generative AI models, such as OpenAI's ChatGPT, has revolutionized how AI is integrated into daily life. These models are becoming more sophisticated, enabling more natural and context-aware interactions.
- **AI in Healthcare:** AI is making significant strides in healthcare, with applications ranging from diagnostics to personalized treatment plans. AI-driven tools are improving patient outcomes and streamlining healthcare operations.
- **AI for Sustainability:** AI technologies are being leveraged to address environmental challenges. This includes optimizing energy consumption, improving waste management, and supporting conservation efforts.
- **Edge AI:** The trend towards edge computing is growing, with AI models being deployed on devices closer to the data source. This reduces latency and enhances real-time decision-making capabilities.
- **AI Ethics and Governance:** As AI becomes more pervasive, there is an increasing focus on ethical considerations and governance frameworks to ensure responsible AI deployment.

**2. Breakthrough AI Technologies in 2024:**
- **Advanced Natural Language Processing (NLP):** NLP technologies have seen significant advancements, enabling more accurate and nuanced understanding of human language. This is enhancing applications in customer service, content creation, and more.
- **AI-Driven Robotics:** Robotics powered by AI are becoming more capable and versatile. These robots are being used in manufacturing, logistics, and even healthcare for tasks such as surgery and rehabilitation.
- **Quantum AI:** The integration of quantum computing with AI is opening new possibilities for solving complex problems that were previously intractable. This includes advancements in drug discovery, cryptography, and optimization problems.
- **AI in Education:** AI is transforming education through personalized learning experiences. AI-driven platforms are providing tailored educational content and real-time feedback to students, enhancing learning outcomes.
- **Autonomous Systems:** AI is driving the development of autonomous systems, including self-driving cars and drones. These systems are becoming more reliable and are expected to have a significant impact on transportation and logistics.

**3. Potential Industry Impacts:**
- **Job Market:** The adoption of AI is expected to have a profound impact on the job market. While some jobs may be displaced, new opportunities are emerging in AI development, data analysis, and AI ethics.
- **Healthcare Industry:** AI is poised to revolutionize healthcare by improving diagnostic accuracy, personalizing treatment plans, and optimizing hospital operations. This could lead to better patient outcomes and reduced healthcare costs.
- **Manufacturing and Logistics:** AI-driven automation is enhancing efficiency in manufacturing and logistics. Predictive maintenance, supply chain optimization, and quality control are some areas benefiting from AI.
- **Finance:** The finance industry is leveraging AI for fraud detection, risk management, and personalized financial services. AI algorithms are improving the accuracy and speed of financial transactions and decision-making.
- **Retail:** AI is transforming the retail sector through personalized shopping experiences, inventory management, and demand forecasting. Retailers are using AI to enhance customer engagement and optimize operations.

**4. AI Applications in Physics:**
- **Coupling AI with Fundamental Physics:** AI is being used to couple with fundamental physics to accelerate calculations and enhance the understanding of physical phenomena. This includes leveraging AI to solve complex equations and model physical systems more efficiently.
- **Generative AI in Physics:** Scientists are using generative AI to answer complex questions in physics, such as mapping out phase diagrams and understanding the mechanisms of physical processes. This approach is driving new paradigms in AI discovery and theoretical investigations.
- **AI in Theoretical Physics:** AI is making significant contributions to theoretical physics by providing new tools for exploring mathematical models and physical theories. This includes advancements in areas such as quantum mechanics and cosmology.